In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import schedule
import time
import json

In [2]:
import sys
sys.path.append("/Users/sunyu/Desktop/crawller/BITNEW/unibit_scraper/History_Data")
import util

In [6]:
def job():
    test_url = "https://finance.yahoo.com/quote/{ticker}/history?p={ticker}"
    crumb_exist = False
    user_crumb = None
    session = None
    # contents = pd.read_csv("test.csv", header=0)
    contents = pd.read_csv("/Users/sunyu/Desktop/crawller/BITNEW/unibit_scraper/History_Data/crypto.csv", header=0)
    contents = pd.DataFrame(data = contents.ticker)
    contents['downloaded'] = np.nan
    contents = contents[0:1]
    today = str(datetime.now().date())
    tomorrow = str(datetime.now().date() + dt.timedelta(days=1))
    yesterday = str(datetime.now().date() - dt.timedelta(days=1))
    # mysql_url ='http://35.222.70.*/api/company/add/stockprice'
    #mysql_url ='http://35.222.70.*/api/stock/crypto'
    #try:
        #while not crumb_exist:
            #session = requests.Session()
            #user_crumb = util.get_crumb(test_url, session)
            #test_data = util.scrape_crypto(session, 'BTC-USD', '2001-01-02', '2018-08-28', user_crumb)
            #if test_data is not None:
                   # crumb_exist = True
    #except Exception as e:
        #print(str(e))

    for index, row in contents.iterrows():
        try:
            session = requests.Session()
            a = requests.adapters.HTTPAdapter(pool_connections =100,pool_maxsize =100,max_retries =3,pool_block= True)# limit size
            b = requests.adapters.HTTPAdapter(pool_connections =100,pool_maxsize =100,max_retries =3,pool_block= True)
            session.mount('http://', b)
            session.mount('https://', a)
            test_url = test_url.format(ticker = row[0])
            user_crumb = util.get_crumb(test_url, session)
        
            if np.isnan(row[1]):
                print("---------------------------")
                ticker_history_data = util.scrape_crypto(session, row[0], '2001-01-01', yesterday, user_crumb)
                if ticker_history_data is None:
                    print("The data does not exist and can not be scraped")
                    contents.loc[index, 'downloaded'] = 0
                else:
                    print("The data exists and can be scraped")
                    ticker_history_data = ticker_history_data.sort_index(ascending=False)
                    for index_history, row_history in ticker_history_data.iterrows():
                        #print(row_history[0])
                        time_stamp = time.mktime(datetime.strptime(str(row_history[0]), "%Y-%m-%d").timetuple())
                        new_cols = {"crypto_name": row[0], "date": row_history[0], "open": row_history[1],
                                    "high": row_history[2], "low": row_history[3], "close": row_history[4],
                                    "adj_close": row_history[5], "volume": row_history[6], "timestamp":int(time_stamp*1000)}
                    
                        util.insert_data_mysql(mysql_url, new_cols)
                        #print(new_cols)
            elif row[1] == 1:
                print("The data was downloaded")
                ticker_history_data = util.scrape_crypto(session, row[0], today, tomorrow, user_crumb)
                if ticker_history_data is None:
                    print("The data of today does not exist")
                else:
                    print("The data of today exists")
                    # new_cols = None
                    for index_history, row_history in ticker_history_data.iterrows():
                        time_stamp = time.mktime(datetime.strptime(str(row_history[0]), "%Y-%m-%d").timetuple())
                        new_cols = {"crypto_name": row[0], "date": row_history[0], "open": row_history[1],
                                    "high": row_history[2], "low": row_history[3], "close": row_history[4],
                                    "adj_close": row_history[5], "volume": row_history[6],"timestamp":int(time_stamp*1000)}
                        util.insert_data_mysql(mysql_url, new_cols)
        except Exception as e:
            print("exception is ",str(e))  
    contents.to_csv("crypto.csv", index=False, sep=',', encoding='utf-8')


if __name__ == "__main__":
    #mysql_url ='http://35.222.70.*/api/stock/crypto'
    # util.insert_data_mysql(mysql_url,{"company_name": "AAPL", "date": "2019-2-19", "open": 12.252, "close": 12.571, "high": 18.677,"low": 11.2345, "adjClose": 12.333, "volume": 123456})
    job()
    # schedule.every().day.at("24:00").do(job)
    # while True:
    #     schedule.run_pending()
    #     time.sleep(1)

---------------------------
https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1=1553572800&period2=1553572800&interval=1d&events=history&crumb=O3403QhkvmT
The data exists and can be scraped
200
